In [ ]:
import numpy as np
import scipy.stats
import pandas as pd
import random as rnd
from gurobipy import *
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Generating affinity matrix, using department partition as shown by Huseyin
rnd.seed(1)
profs = 30 #num professors
num_departments = 5 #num departments
preference_prob = .5 #prob of professor i within department d liking professor j within same department
 
C = np.zeros((profs, profs))

rnd.seed(1)
#partitions = np.random.randint(0,profs,size = num_departments - 1)
partitions = [5, 9, 16, 23]
partitions.sort()
partitions = np.append(partitions,profs)


# last_p = 0
# for p in partitions:
#     C[last_p:p,last_p:p] = np.random.binomial(1,preference_prob, size = (p - last_p,p - last_p))
#     last_p = p

last_p = 0
for p in partitions:
    for prof in range(last_p, p):
        affinities = np.random.randint(last_p,p,size = 2)
        C[prof, affinities] = 1
    last_p = p

for i in range(0,profs):
    C[i,i] = 1
    
print(partitions)
print(C)

In [ ]:
#generating room preferences

#hard-coded room numbers
second_floor_rooms = [210,212,214,216,220,252,254,255,
                      256,257,259,261,262,263, 264,265,
                      266,268,270,271,273, 291, 293]
    
third_floor_rooms = [int('3' + str(num)[1:]) for num in second_floor_rooms]
third_floor_rooms.remove(391) # remove 391 393
third_floor_rooms.remove(393)
room_list = second_floor_rooms + third_floor_rooms
rooms = len(room_list)
shuffled_rooms = rnd.sample(room_list, rooms)
num_rankings = 3

#R_ is num_professors x num_ranking matrix, R_ik is the kth choice room number for professor i
R_ = np.zeros((profs,num_rankings))

#uniform case: generates room rankings for each professor, assuming all rooms are equally preferred 
# for i in range(profs):
#     R_[i] = np.random.choice(room_list, size=num_rankings, replace=False, p=None)

#weighted case: generates room rankings for each professor, assuming room preferences follow a geometric distribution ~geom(p)  
p = .2
room_probs = [scipy.stats.geom.pmf(k,p) for k in range(1,rooms+1)]
room_probs[-1] = 1 - min(sum(room_probs[:-1]),1)

for i in range(profs):
    R_[i] = np.random.choice(shuffled_rooms, size=num_rankings, replace=False, p=room_probs)
    print(R_[i])
    
#R is num_professors x num_room X num_ranking matrix, 
#R_ijk is 1 if professor i's kth ranked room is the jth element of room_list
R= np.zeros((profs,rooms,num_rankings))
for i in range(profs):
    for j in range(rooms):
        for k in range(num_rankings):
            if R_[i,k]==shuffled_rooms[j]:
                R[i,j,k]=1
R

In [ ]:
floors = 2
lowest_floor = 2
#F is num_professors X num floors matrix where F_il = 1 if professor i wants to sit on floor l
F = np.zeros((profs,floors))
#if professor i chose a room on floor l in his rankings R_, then we assume she wants to sit on that floor
for i in range(profs):
    floor_nums_for_i = [int(str(n)[0]) - lowest_floor for n in R_[i]]
    for l in floor_nums_for_i:
        F[i,l] = 1
F

In [ ]:
#import the distance matrix

Distance = pd.read_csv("huddle_distance_csv.csv", index_col= 0, sep=";")
Distance.fillna(value=0, inplace=True)

D = np.array(Distance)


# D = np.zeros( ( rooms , rooms ) )
# for i in range( rooms ):
#     for j in range( i ):
#         if i==j:
#             D[ i ][ j ] = 0
#         elif room_list[i]//100 != room_list[j]//100: # BE CAREFUL!! PYTHON 3 FLOOR DIVISION
#             D[ i ][ j ] = rnd.randint( 101 , 110 )
#             D[j][i] = D[i][j]
#         else:
#             D[ i ][ j ] = rnd.randint( 1 , 10 )
#             D[j][i] = D[i][j]
            
M = np.array([8,4,2])
d = 4
D.shape

In [ ]:
Distance.loc[210,'261']

In [ ]:
# def initializeData():
#     rnd.seed( 1 )
#     room_no = 12
#     prof_no = 10
#     D = np.zeros( ( room_no , room_no ) )
#     for i in range( room_no ):
#         for j in range( room_no ):
#             if i==j:
#                 D[ i ][ j ] = 0
#             elif i%100 != j%100:
#                 D[ i ][ j ] = rnd.uniform( 100 , 110 )
#             else:
#                 D[ i ][ j ] = rnd.uniform( 0 , 10 )
    
#     #Affinity = np.array([[1,1,0,0,0],[1,1,0,0,0],[0,0,1,0,0],[0,1,1,1,0],[0,1,0,0,1]])
    
#     #floors preference
#     return(D)

# def distance( i , j ):
#     global facs , custs
#     dx = facs[ i ][ 0 ] - custs[ j ][ 0 ]
#     dy = facs[ i ][ 1 ] - custs[ j ][ 1 ]
#     val = ( ( dx * dx ) + ( dy * dy ) ) ** 0.5
#     return val

def constructVars():
    global profs , rooms , num_rankings , myModel , xVar
    for i in range( profs ):
        for j in range(rooms):
            xVar[i][j] = myModel.addVar( vtype = GRB.BINARY , ub = 1 , name = "x" + str( i ) + "_" + str(room_list[j]) )
    myModel.update()
    
def constructObj():
    global profs, rooms, num_rankings , myModel, xVar
    objExpr = LinExpr()
    for i in range( profs):
        for k in range(num_rankings):
            for j in range( rooms ):
                objExpr += M[k] * R[i][j][k] * xVar[i][j]
    myModel.setObjective( objExpr , GRB.MAXIMIZE )
    
def constructConstrs():
    global profs, rooms, num_rankings, floors, myModel, xVar
    constrExpr = LinExpr()
    for i in range( profs ):
        for l in range( floors ):  # be careful with where it starts and ends
            constrExpr = LinExpr()
            for j in range( rooms ):
                if room_list[j]>= 100*(l+lowest_floor) and room_list[j]<100*(l+lowest_floor+1):        
                    constrExpr += xVar[ i ][ j ]
            myModel.addConstr( lhs = constrExpr , sense = GRB.LESS_EQUAL , rhs = F[i][l] , name = "prof_" + str( i ) + "_floor_" + str(l+lowest_floor))
    
    for j in range( rooms ):
        constrExpr = LinExpr()
        for i in range( profs ):
            constrExpr += xVar[ i ][ j ]
        myModel.addConstr( lhs = constrExpr, sense = GRB.LESS_EQUAL , rhs = 1 , name= "room_" + str( room_list[j] ) + "_assigned_max_once" )
    
    for i in range( profs ):
        constrExpr = LinExpr()
        for j in range( rooms ): 
            constrExpr += xVar[ i ][ j ]
        myModel.addConstr( lhs = constrExpr, sense = GRB.EQUAL , rhs = 1 , name= "prof_" + str( i ) + "_gets_a_room" )
    
    indices = np.transpose(np.nonzero(C))
    for index in indices:
        if index[0] != index[1]:
            for j in range(rooms):
                for j_prime in range( rooms ): # doesn't need to run through all the rooms, we will have duplicate constraints
                    constrExpr = LinExpr()
                    constrExpr += D[j][j_prime]*(xVar[index[0]][j] + xVar[index[1]][j_prime] - 1)
                    myModel.addConstr( lhs = constrExpr, sense = GRB.LESS_EQUAL , rhs = d , name= "Affinity_prof_" + str( index[0] ) + "_room_" + str(room_list[j]) + "_with_prof_" + str(index[1]) +"_room_"+str(room_list[j_prime]))
    myModel.update()

In [ ]:
### model based on objective function (room & affinity preference)

def constructVars():
    global profs , rooms , num_rankings , myModel , xVar
    for i in range( profs ):
        for j in range(rooms):
            xVar[i][j] = myModel.addVar( vtype = GRB.BINARY , ub = 1 , name = "x" + str( i ) + "_" + str(room_list[j]) )
    myModel.update()
    
def constructObj():
    global profs, rooms, num_rankings , myModel, xVar
    objExpr = LinExpr()
    for i in range( profs):
        for k in range(num_rankings):
            for j in range( rooms ):
                objExpr += M[k] * R[i][j][k] * xVar[i][j]
    myModel.setObjective( objExpr , GRB.MAXIMIZE )
    
def constructConstrs():
    global profs, rooms, num_rankings, floors, myModel, xVar
    constrExpr = LinExpr()
    for i in range( profs ):
        for l in range( floors ):  # be careful with where it starts and ends
            constrExpr = LinExpr()
            for j in range( rooms ):
                if room_list[j]>= 100*(l+lowest_floor) and room_list[j]<100*(l+lowest_floor+1):        
                    constrExpr += xVar[ i ][ j ]
            myModel.addConstr( lhs = constrExpr , sense = GRB.LESS_EQUAL , rhs = F[i][l] , name = "prof_" + str( i ) + "_floor_" + str(l+lowest_floor))
    
    for j in range( rooms ):
        constrExpr = LinExpr()
        for i in range( profs ):
            constrExpr += xVar[ i ][ j ]
        myModel.addConstr( lhs = constrExpr, sense = GRB.LESS_EQUAL , rhs = 1 , name= "room_" + str( room_list[j] ) + "_assigned_max_once" )
    
    for i in range( profs ):
        constrExpr = LinExpr()
        for j in range( rooms ): 
            constrExpr += xVar[ i ][ j ]
        myModel.addConstr( lhs = constrExpr, sense = GRB.EQUAL , rhs = 1 , name= "prof_" + str( i ) + "_gets_a_room" )
    
    indices = np.transpose(np.nonzero(C))
    for index in indices:
        if index[0] != index[1]:
            for j in range(rooms):
                for j_prime in range( rooms ): # doesn't need to run through all the rooms, we will have duplicate constraints
                    constrExpr = LinExpr()
                    constrExpr += D[j][j_prime]*(xVar[index[0]][j] + xVar[index[1]][j_prime] - 1)
                    myModel.addConstr( lhs = constrExpr, sense = GRB.LESS_EQUAL , rhs = d , name= "Affinity_prof_" + str( index[0] ) + "_room_" + str(room_list[j]) + "_with_prof_" + str(index[1]) +"_room_"+str(room_list[j_prime]))
    myModel.update()

In [ ]:
## integer programming optimization
# ( noFacs , noCusts , facs , custs , fixs , caps , dems ) = initializeData()
myModel = Model( "seat assignment" )
# yVars = [ 0 for i  in range( noFacs ) ]
# xVars = [ [ 0 for j in range ( noCusts ) ] for i in range ( noFacs ) ]
xVar = np.empty([profs,rooms], dtype=Var)
constructVars()
constructObj()
constructConstrs()
myModel.write( "seat_assignment.lp" )
myModel.optimize()
print('optimal objective: {}'.format(myModel.ObjVal))
print('optimal solution:')
allVars = myModel.getVars()
for curVar in allVars:
    if curVar.x == 1:
        print(curVar.varName + '  ' + str(curVar.x))

In [ ]:
room_list

In [ ]:
np.transpose(np.nonzero(C))